참고 git: https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco-pose.yaml

# YOLOv8 Pose

mAP50: 80.1   
mAP50-95: 50.4

In [21]:
from ultralytics import YOLO
model = YOLO('yolov8n-pose.pt')

In [24]:
yolo_results

ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000029F9A4AF3D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,

In [23]:
# 평가할 데이터셋 경로
data_yaml = r'C:\Users\705-18\Desktop\a\data.yaml'
# mAP 계산
yolo_results = model.val(data=data_yaml)  # val() 메서드 사용하여 평가
print(dir(yolo_results))

Ultralytics 8.3.9  Python-3.8.19 torch-1.10.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\705-18\Desktop\a\datasets\coco-pose\labels\val2017.cache... 2346 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2346/2346 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 147/147 [00:31<00:00,  4.66it/s]


                   all       2346       6352      0.866      0.826      0.912        0.7      0.837      0.746      0.798      0.511
Speed: 0.2ms preprocess, 5.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Saving runs\pose\val\predictions.json...

Evaluating pycocotools mAP using runs\pose\val\predictions.json and C:\Users\705-18\Desktop\a\datasets\coco-pose\annotations\person_keypoints_val2017.json...
loading annotations into memory...
Done (t=0.51s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.03s).
Accumulating evaluation results...
DONE (t=0.79s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.540
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.733
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.599
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= 

# Detectron2 Keypoints rcnn

In [1]:
import detectron2
import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
import matplotlib.pyplot as plt

c:\Users\705-18\anaconda3\envs\detectron\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE='cuda' #만약 gpu를 사용한다면 ‘cuda’로 수정
predictor = DefaultPredictor(cfg)

In [3]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

# COCO 데이터셋 경로 설정
val_dataset_dir = r"C:\Users\705-18\Desktop\a\datasets\coco-pose\images\val2017"
val_annotations_path = r"C:\Users\705-18\Desktop\a\datasets\coco-pose\annotations\person_keypoints_val2017.json"

# COCO 데이터셋 등록 (val)
register_coco_instances("coco_val", {}, val_annotations_path, val_dataset_dir)

# 메타데이터 로드
coco_val_metadata = MetadataCatalog.get("coco_val")

# 데이터셋 확인 (optional)
val_dataset_dicts = DatasetCatalog.get("coco_val")

In [4]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# COCOEvaluator 설정
evaluator = COCOEvaluator("coco_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "coco_val")

# mAP 계산 (inference_on_dataset 함수 사용)
results = inference_on_dataset(predictor.model, val_loader, evaluator)

# mAP 결과 출력
print(results)

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
c:\Users\705-18\anaconda3\envs\detectron\lib\site-packages\detectron2\structures\image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
c:\Users\705-18\anaconda3\envs\detectron\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
c:\Users\705-18\anaco

Loading and preparing results...
DONE (t=0.18s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.533
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.780
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.590
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.618
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.718
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.192
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.549
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.677
 Average Recall     (AR) @[ IoU=0.50:0.

In [16]:
len(results['bbox'])

6

In [18]:
print("mAP50: ", results["bbox"]["AP50"])

mAP50:  77.97202266637228


In [19]:
# YOLO 형식으로 mAP 결과 출력
mAP_50 = results["bbox"]["AP50"]
mAP_50_95 = results["bbox"]["AP"]

print(f"mAP@0.5: {mAP_50:.3f}")
print(f"mAP@0.5:0.95: {mAP_50_95:.3f}")

mAP@0.5: 77.972
mAP@0.5:0.95: 53.255
